In [58]:
import sys
import os

# Add the parent directory to the Python path
parent_dir = os.path.abspath('../..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from resynthesis import resynthesis_from_model
import torch
import librosa
from IPython.display import Audio, display
import soundfile as sf

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# synthesis_parameters = {
#     "bubbles"  : [0.25, 1, 0.30],
#     "fire"     : [0,    1, 0],
#     "keyboard" : [0.25, 1, 0.33],
#     "rain"     : [0.25,   0, 0],
#     "river"    : [0.25, 1, 0.75],
#     "shards"   : [0,    1, 0.5],
#     "waterfall": [0.33, 1, 1],
#     "wind"       : [1,    1, 0.25]
# }

# def full_synthesis(texture_type, input_audio_improver, energy_imposition, envelope_follower):
#     model_paths = "checkpoints/"
#     sound_paths = "all_sounds/"

#     audio_path = os.path.join(sound_paths, f"{texture_type}.wav")
#     original   = librosa.load(audio_path, sr=44100, mono=True)[0]
#     print(f"Original sound: {texture_type}")
#     display(Audio(original, rate=44100, normalize=True))
#     # Make a list of all folders in model_paths that start with texture_type
#     model_folders = [f for f in os.listdir(model_paths) if f.startswith(texture_type)]
#     model_folders.sort()
#     for model_folder in model_folders:
#         model_folder_path = os.path.join(model_paths, model_folder)
#         print(f"Processing {audio_path} with model {model_folder_path}")
#         resynthesis = resynthesis_from_model(audio_path, model_folder_path, input_audio_improver, energy_imposition, envelope_follower).detach().cpu().numpy()
#         sf.write(model_folder+'.wav', resynthesis, 44100)
#         print(f"Resynthesized sound: {model_folder}")
#         display(Audio(resynthesis, rate=44100, normalize=True))

# texture_types = synthesis_parameters.keys()
# # Iterate over all texture types
# for texture_type_output in texture_types:
#     # Get the synthesis parameters for the current texture type
#     input_audio_improver, energy_imposition, envelope_follower = synthesis_parameters[texture_type_output]
#     # Perform the full synthesis
#     full_synthesis(texture_type_output, input_audio_improver, energy_imposition, envelope_follower)

In [61]:
import os

import soundfile as sf

def save_audio(audio_data, output_path, sample_rate=44100):
    """
    Save audio using librosa-compatible soundfile.
    Assumes audio_data is a 1D numpy array with float32 values between -1 and 1.
    """
    sf.write(output_path, audio_data, samplerate=sample_rate)

# Timbre transfer synthesis parameters
synthesis_parameters = {
    "bubbles"  : [0.25, 1, 0.30],
    "fire"     : [0,    1, 0],
    "keyboard" : [0.25, 1, 0.33],
    "rain"     : [0.25,   0, 0],
    "river"    : [0.25, 1, 0.75],
    "shards"   : [0,    1, 0.5],
    "waterfall": [0.33, 1, 1],
    "wind"     : [1,    1, 0.25]
}

# Define texture types and model paths
texture_types = synthesis_parameters.keys()
models_path = "checkpoints/"
input_folder = "all_sounds/"

# Ensure models_path exists
if not os.path.exists(models_path):
    raise FileNotFoundError(f"Models path '{models_path}' does not exist.")

# Process each texture type as the output
for texture_type_output in texture_types:
    print(f"Processing timbre transfer for output texture: {texture_type_output}")
    
    # Get synthesis parameters for the current output texture type
    input_audio_improver, energy_imposition, envelope_follower = synthesis_parameters[texture_type_output]
    
    # Get the model folders that start with the current texture type
    model_folders = [f for f in os.listdir(models_path) if f.startswith(texture_type_output)]
    
    if not model_folders:
        print(f"No models found for texture type: {texture_type_output}")
        continue
    
    model_folders.sort()  # Sort model folders

    # Iterate over all models for the current output texture type
    for i, model_folder in enumerate(model_folders, start=1):
        model_folder_path = os.path.join(models_path, model_folder)
        print(f"Using model folder: {model_folder_path}")

        # Iterate over all texture types for the input (you may want this for diversity)
        for texture_type_input in texture_types:
            input_audio_path = os.path.join(input_folder, f"{texture_type_input}.wav")
            
            # Check if the input file exists
            if not os.path.exists(input_audio_path):
                print(f"Warning: Input file '{input_audio_path}' not found. Skipping this texture type.")
                continue
            
            print(f"Processing timbre transfer from '{texture_type_input}' to '{texture_type_output}' using model '{model_folder}'")

            # Call the resynthesis function (ensure it returns a tensor or numpy array)
            try:
                resynthesis = resynthesis_from_model(input_audio_path, model_folder_path, input_audio_improver, energy_imposition, envelope_follower)
                resynthesis = resynthesis.detach().cpu().numpy()  # Convert to numpy array

                # Prepare the output path for saving the resynthesis
                output_path = os.path.join(f"{texture_type_input}_to_{texture_type_output}_{i}.wav")
                print(f"Saving resynthesized audio to: {output_path}")

                # Save the resynthesis (you might need a method to save numpy array to wav)
                save_audio(resynthesis, output_path)  # Make sure save_audio is defined elsewhere

            except Exception as e:
                print(f"Error during resynthesis: {e}")

    print(f"Finished processing for output texture: {texture_type_output}")

Processing timbre transfer for output texture: bubbles
Using model folder: checkpoints/bubbles_1
Processing timbre transfer from 'bubbles' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: bubbles_to_bubbles_1.wav
Processing timbre transfer from 'fire' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: fire_to_bubbles_1.wav
Processing timbre transfer from 'keyboard' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: keyboard_to_bubbles_1.wav
Processing timbre transfer from 'rain' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: rain_to_bubbles_1.wav
Processing timbre transfer from 'river' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: river_to_bubbles_1.wav
Processing timbre transfer from 'shards' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio to: shards_to_bubbles_1.wav
Processing timbre transfer from 'waterfall' to 'bubbles' using model 'bubbles_1'
Saving resynthesized audio t